# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [6]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [7]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


FileNotFoundError: [Errno 2] No such file or directory: 'reviews_data.txt.gz'

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [ ]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [9]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

NameError: name 'documents' is not defined

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [5]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


2020-08-29 11:01:15,433 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8744967579841614),
 ('unclean', 0.7758036255836487),
 ('stained', 0.7711732387542725),
 ('smelly', 0.7667703032493591),
 ('dusty', 0.7639738321304321),
 ('grubby', 0.7565650939941406),
 ('dingy', 0.7385347485542297),
 ('grimy', 0.7245001792907715),
 ('soiled', 0.7236559391021729),
 ('gross', 0.7206484079360962)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [6]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)

[('courteous', 0.921128511428833),
 ('friendly', 0.8279157876968384),
 ('cordial', 0.801971971988678),
 ('professional', 0.7868619561195374),
 ('curteous', 0.7700151205062866),
 ('attentive', 0.7695232629776001)]

In [7]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)

[('germany', 0.694383442401886),
 ('canada', 0.6543870568275452),
 ('midwest', 0.630740225315094),
 ('barcelona', 0.6271055340766907),
 ('rome', 0.6205963492393494),
 ('spain', 0.6154872179031372)]

In [8]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)

[('horrified', 0.8131061792373657),
 ('amazed', 0.8086591958999634),
 ('astonished', 0.7797779440879822),
 ('dismayed', 0.7650278806686401),
 ('astounded', 0.7474824786186218),
 ('appalled', 0.7402036786079407)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [9]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)

[('duvet', 0.7114425301551819),
 ('mattress', 0.6982126832008362),
 ('pillowcase', 0.6902005672454834),
 ('blanket', 0.6881498098373413),
 ('matress', 0.6708730459213257),
 ('quilt', 0.6524391770362854),
 ('sheets', 0.636884331703186),
 ('pillows', 0.6267510056495667),
 ('foam', 0.6152631044387817),
 ('comforter', 0.6020686626434326)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [10]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.76677036

In [11]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [12]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.26689175

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [13]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

/usr/local/lib/python3.8/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'france'

In [14]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Saving the model to a file

Use `<model>.save(<file name>)` to save model to a file

Use `<model> = KeyedVectors.load(<file name>)` to save model to a file

In [ ]:
model.save('review.model.bin')
from gensim.models import KeyedVectors
model2 = KeyedVectors.load('review.model.bin')


In [26]:
model2.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

In [28]:
model.wv.save_word2vec_format('review.model.txt', binary=False)

2020-08-29 11:18:11,521 : INFO : storing 70537x150 projection weights into review.model.txt


In [29]:
!head -2 review.model.txt

70537 150
the -1.0474149 -1.0775484 0.75805414 0.7357467 -0.65385985 -1.2986574 0.8246915 -0.9524938 -2.5136108 -0.004296381 1.9257911 1.5469992 -1.6204963 2.8012848 0.04588041 1.2353007 0.61977947 1.0574156 0.44046468 0.5489067 0.6377579 0.5728992 1.5188081 -1.8405206 0.16099833 0.52852774 0.90188515 1.2018433 1.2751708 -0.35817325 0.5333824 1.3552257 1.0180264 -0.32291555 0.33117554 -0.7084595 0.44710004 0.48289528 0.68740976 0.13262022 1.4459572 0.39889675 0.5991782 1.6264985 0.2768606 -1.2750913 1.2704366 -0.6263162 -3.2955897 -0.2677531 1.4692366 -1.802373 -0.03654402 -0.13222122 -0.5125803 -1.1268432 -0.65204334 -0.9096653 1.3756621 0.6438835 -2.1022336 1.5402129 -2.6617522 -0.8364755 -0.005132524 -2.0893056 0.42927772 -0.70986706 -0.8546314 -1.3894821 -0.9202516 -0.7117637 -1.2846073 -0.58542544 1.5928258 0.48895657 1.2392563 -1.5895292 1.7774339 -0.95572954 -0.69329685 -0.8601518 -2.792009 -0.3416773 0.927962 0.47721827 -0.8427079 -0.047970477 -1.4401464 -1.958336 1.7821808 -0.

In [30]:
!tail -2 review.model.txt

longmen -0.04464475 0.013499864 0.017939413 0.11016689 -0.035444584 0.019699477 0.11995376 0.0068000094 0.07599533 -0.04476829 0.016287733 -0.036519367 0.048385922 -0.05286206 0.038155362 0.173763 0.06538556 -0.041321516 -0.118958965 -0.16379856 0.04313128 0.102543525 0.030428315 0.03488412 0.023136787 -0.07679798 0.058719795 -0.037489478 0.01734984 0.040495973 -0.029823137 -0.035882834 -0.12941591 0.094231255 -0.13206856 0.005374585 0.062290065 0.019999018 0.077333644 0.048102587 0.013850405 -0.004091791 -0.06944274 0.043887604 0.0037073672 0.075675584 0.18738243 0.04102377 -0.07842261 0.03169691 0.04388537 0.0748127 0.048892267 -0.06649639 0.051919885 -0.043634724 -0.023626395 0.018510515 -0.094644316 -0.015341737 0.024523273 0.093875326 0.036184747 0.100009866 0.104307674 -0.11368899 -0.010508096 0.0053770808 0.23867266 -0.012560841 -0.080248356 0.009368075 -0.13071427 -0.04406637 -0.048579864 -0.067995474 0.023842273 -0.07534184 0.0019154936 -0.03527242 0.012816708 -0.026099375 -0.

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
